In [1]:
from configparser import ConfigParser
import psycopg2

In [2]:
def config(filename='database.ini', section='postgresql'):
    parser = ConfigParser()
    parser.read(filename)

    db_params = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db_params[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db_params

In [3]:
def connect(function, *func_params):
    
    params = config()
    connection = psycopg2.connect(**params)
    global cur
    cur = connection.cursor()
        
    function(*func_params)

    cur.close()
    connection.close()


In [20]:
def get_table_list(t_schema="public"):
    s = ""
    s += "SELECT"
    s += " table_schema"
    s += ", table_name"
    s += " FROM information_schema.tables"
    s += " WHERE"
    s += " ("
    s += " table_schema = '" + t_schema + "'"
    s += " )"
    s += " ORDER BY table_schema, table_name;"

    cur.execute(s)
    list_tables = cur.fetchall()

    for t_name_table in list_tables:
        print(t_name_table[1])

In [21]:
connect(get_table_list)

auth_group
auth_group_permissions
auth_permission
auth_user
auth_user_groups
auth_user_user_permissions
botsite_userprofile
django_admin_log
django_content_type
django_migrations
django_session
vk_chat
vk_chatmessage
vk_intervalphrase
vk_kicknonmemberssetting
vk_newpostsetting
vk_randompostsetting
vk_smartreply
vk_usermessage


In [28]:
def get_data_from_table(table_name):
    s = f"SELECT * FROM {table_name}"
    cur.execute(s)
    table_data = cur.fetchall()
    print(table_data)
    return table_data

In [31]:
connect(get_data_from_table, "vk_chat")

[(2000000001, 1, None, 3, 21070693, False, False, None, None, datetime.datetime(2019, 11, 18, 16, 20, 37, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=180, name=None)), datetime.datetime(2020, 10, 4, 6, 16, 4, 551460, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=180, name=None)), False), (2000000004, 4, None, 4, 4003081, True, False, None, None, datetime.datetime(2019, 12, 6, 14, 36, 32, 324086, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=180, name=None)), datetime.datetime(2020, 1, 9, 8, 32, 17, 350115, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=180, name=None)), False), (2000000005, 5, None, 2, 21070693, True, False, None, None, datetime.datetime(2019, 12, 7, 17, 3, 3, 462937, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=180, name=None)), datetime.datetime(2020, 11, 21, 16, 17, 41, 973432, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=180, name=None)), False), (2000000003, 3, None, 7, 21070693, True, False, None, None, datetime.datetime(2019, 11, 27, 18, 3, 28, tzinfo=psycopg

In [30]:
connect(get_data_from_table, "v_chat")

UndefinedTable: ОШИБКА:  отношение "v_chat" не существует
LINE 1: SELECT * FROM v_chat
                      ^


In [35]:
def get_record_from_table(table_name, column_name, column_value):
    # есть ли такая таблица
    # есть ли такая колонка
    # есть ли такое валью
    # является ли она юник
    # если не юник, выводим предупреждение, что возьмется первая строка
    s = f"SELECT * FROM {table_name} where {column_name} = {column_value}"
    cur.execute(s)
    table_data = cur.fetchall()
    print(table_data)
    return table_data

In [36]:
connect(get_record_from_table, "vk_chat", "chat_id", 2000000001)

[(2000000001, 1, None, 3, 21070693, False, False, None, None, datetime.datetime(2019, 11, 18, 16, 20, 37, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=180, name=None)), datetime.datetime(2020, 10, 4, 6, 16, 4, 551460, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=180, name=None)), False)]


In [42]:
def check_if_column_unique_primary(table_name, column_name):
        
    s = f"""SELECT count(*) > 0 
    FROM information_schema.key_column_usage AS c
    LEFT JOIN information_schema.table_constraints AS t
    ON t.constraint_name = c.constraint_name
    WHERE t.constraint_type in ('PRIMARY KEY', 'UNIQUE') 
    and t.table_name = '{table_name}' and c.column_name = '{column_name}';

    """
    cur.execute(s)
    result = cur.fetchone()[0]
    if result is False:
        print(result)
        print(type(result))
        return result
        

In [45]:
connect(check_if_column_unique_primary, "vk_cha", "chat_id")

False
<class 'bool'>


In [47]:
def get_unique_columns(table_name):
    s = f"""SELECT c.column_name 
    FROM information_schema.key_column_usage AS c
    LEFT JOIN information_schema.table_constraints AS t
    ON t.constraint_name = c.constraint_name
    WHERE t.constraint_type in ('PRIMARY KEY', 'UNIQUE') 
    and t.table_name = '{table_name}';

    """
    cur.execute(s)
    result = tuple(i[0] for i in cur.fetchall())
    return result
    

In [48]:
connect(get_unique_columns, "auth_user",)

('username', 'id')


In [46]:
def connect(function, *func_params):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        params = config()

        # connect to the PostgreSQL server
        conn = psycopg2.connect(**params)

        global cur
        cur = conn.cursor()
        
        function(*func_params)

        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()


[(2000000001, 1, None, 3, 21070693, False, False, None, None, datetime.datetime(2019, 11, 18, 16, 20, 37, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=180, name=None)), datetime.datetime(2020, 10, 4, 6, 16, 4, 551460, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=180, name=None)), False)]


In [61]:
def get_record_from_table(table_name, column_name, value):
    # есть ли такая таблица
    # есть ли такая колонка
    # есть ли такое валью
    # является ли она юник
    # если не юник, выводим предупреждение, что возьмется первая строка
    s = f"SELECT * FROM {table_name} where {column_name} = {value}"
    cur.execute(s)
    table_data = cur.fetchall()
    if not table_data:
        raise ValueError(f"ERROR: there is no value '{value}' in column '{column_name}'")
    if len(table_data) > 0:
         raise ValueError("ERROR: there is more than one value for your request. Use columns with unique constraits.")
        
    return table_data[0]

In [62]:
connect(get_record_from_table, "vk_chatmessage", "chat_id_id", 2000000001)

ERROR: there is more than one value for your request.


In [4]:
def get_column_names(table_name):
    s = f"""SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table_name}';"""
    cur.execute(s)
    column_names = cur.fetchall()
    
    return tuple(item[0] for item in column_names)

In [5]:
connect(get_column_names, "vk_chat")

[('chat_id',), ('local_id',), ('title',), ('members_count',), ('owner_id',), ('conversation_is_registered',), ('interval_mode',), ('interval',), ('messages_till_endpoint',), ('first_contact',), ('last_contact',), ('smart_mode',)]


[(2000000001, 1, None, 3, 21070693, False, False, None, None, datetime.datetime(2019, 11, 18, 16, 20, 37, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=180, name=None)), datetime.datetime(2020, 10, 4, 6, 16, 4, 551460, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=180, name=None)), False)]

In [21]:
my_list = [('chat_id',), ('local_id',), ('title',), ('members_count',), ('owner_id',), ('conversation_is_registered',), ('interval_mode',), ('interval',), ('messages_till_endpoint',), ('first_contact',), ('last_contact',), ('smart_mode',)]
list2 = tuple(item[0] for item in my_list)

In [22]:
list1

(('chat_id',
  'local_id',
  'title',
  'members_count',
  'owner_id',
  'conversation_is_registered',
  'interval_mode',
  'interval',
  'messages_till_endpoint',
  'first_contact',
  'last_contact',
  'smart_mode'),)

In [26]:
list2

('chat_id',
 'local_id',
 'title',
 'members_count',
 'owner_id',
 'conversation_is_registered',
 'interval_mode',
 'interval',
 'messages_till_endpoint',
 'first_contact',
 'last_contact',
 'smart_mode')